In [1]:
import pandas as pd
import os
from tqdm import tqdm

In [9]:
root = '/shared/projects/mycovirus/'

In [10]:
gene_name = pd.read_csv(f'{root}Af293_geneName.tsv',on_bad_lines='skip', sep='\t')

In [11]:
gene_name.drop(['Common Name','Taxonomic Name'], axis=1, inplace=True)
gene_name.head(2)

,NCBI GeneID,Symbol,Description
0,3509526,AFUA_4G06890,14-alpha sterol demethylase Cyp51A
1,3513640,AFUA_2G07680,L-ornithine N5-oxygenase SidA


In [21]:
def significatives_genes_export(path, file, out):
    df = pd.read_csv(path, sep=';', index_col=0)

    for col in df.columns:
        df[col] = df[col].str.replace(',', '.', regex=False)
        df[col] = df[col].astype('float')

    subset = df[df['pvalue'] < 0.05][abs(df['log2FoldChange'])>1.5]

    subset['Symbol'] = [i.split(':')[1] for i in subset.index]

    merge = pd.merge(gene_name, subset, on='Symbol', how='inner')

    merge.to_csv(f'{out}{file}', index=False)
    print(f'{file} : {merge.shape}')

In [22]:
files = [i for i in os.listdir(f'{root}clustering/deseq/') if i.endswith('.csv')]
files

['k3_3_vs_1.csv', 'k2_1_vs_2.csv', 'k3_2_vs_1.csv', 'k3_3_vs_2.csv']

In [23]:
out = f'{root}genes_significatives/clustering/'
for file in files:
    path = f'{root}clustering/deseq/{file}'
    significatives_genes_export(path, file, out)

/shared/software/miniconda/envs/python-pytorch-tensorflow-3.7-1.11.0-2.6.0/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


k3_3_vs_1.csv : (292, 9)
k2_1_vs_2.csv : (194, 9)
k3_2_vs_1.csv : (189, 9)
k3_3_vs_2.csv : (48, 9)


In [85]:
merge['Description'].value_counts()

hypothetical protein                             87
MFS multidrug transporter, putative               6
C6 transcription factor, putative                 3
aminotransferase, putative                        2
ThiJ/PfpI family protein                          2
                                                 ..
beta-galactosidase, putative                      1
DUF1275 domain protein                            1
4-hydroxyphenylpyruvate dioxygenase, putative     1
serine protein kinase, putative                   1
tRNA-Phe                                          1
Name: Description, Length: 156, dtype: int64

### Merge metadata with k mean clusters

In [24]:
col = pd.read_csv(f'{root}results/colData.csv')
kmean = pd.read_csv(f'{root}clustering/kmean_clustering.csv', sep=';').rename(columns = {'Unnamed: 0':'sampleID'})

In [26]:
merge = pd.merge(col, kmean, on='sampleID', how='inner')

In [29]:
merge.to_csv("/shared/projects/mycovirus/rnaseq_metadata_kmean.csv")